In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                        transforms.ToTensor(), # convert to pytorch tensor
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                        ])

In [3]:
root = 'data/ShanghaiTech_Crowd_Counting_Dataset/'

In [6]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test] # to perform prediction on test_data 

In [7]:
path_sets

['data/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/images']

In [8]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

### Call CSRNet model
It willl download imagenet pretrained vgg16 model to system path (/root/.torch/models)

In [9]:
model = CSRNet()

In [10]:
type(model)

model.CSRNet

In [23]:
#print(str(model))

In [11]:
model = model.cuda() # move to GPU CUDA
# model.to('cuda:1')

In [11]:
!ls data/models

PartAmodel_best.pth.tar  partBmodel_best.pth.tar


In [12]:
# checkpoint = torch.load('model_best.pth.tar')
checkpoint = torch.load('data/models/PartAmodel_best.pth.tar') # load pth weight file

In [14]:
checkpoint.keys()

['arch', 'epoch', 'state_dict', 'optimizer', 'best_prec1']

In [16]:
print(checkpoint['arch'])

None


In [17]:
print(checkpoint['epoch'])

376


In [19]:
type(checkpoint['state_dict']) # weights

collections.OrderedDict

In [21]:
# checkpoint['state_dict'].keys()

In [22]:
print(checkpoint['best_prec1'])

tensor(65.9016, device='cuda:0')


In [24]:
len(img_paths)

182

In [13]:
mae = 0
# Step through image one-by-one, it would be more efficient to use a data loader (batch-wise)
for i in xrange(len(img_paths)): # 182 test images 
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB')) # scale to [0..255]
    # subtract the dataset image mean
    img[0,:,:]=img[0,:,:]-92.8207477031 
    img[1,:,:]=img[1,:,:]-95.2757037428
    img[2,:,:]=img[2,:,:]-104.877445883
    img = img.cuda() # move input to GPU CUDA
    #img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')#load gt h5 file
    groundtruth = np.asarray(gt_file['density']) # get gt density map
    output = model(img.unsqueeze(0)) # add one 
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print i,mae


0 291.0068664550781
1 954.1450500488281
2 1396.2145080566406
3 1719.8500061035156
4 1959.8150024414062
5 2513.0576171875
6 3775.0765380859375
7 4183.470703125
8 5495.567626953125
9 5836.9293212890625
10 6157.234558105469
11 6692.102722167969
12 7898.266052246094
13 9057.491394042969
14 9597.601440429688
15 9983.790710449219
16 10122.806640625
17 10368.40788269043
18 11590.929611206055
19 11911.221115112305
20 13586.018112182617
21 13935.525527954102
22 14360.905960083008
23 14775.20344543457
24 14954.519912719727
25 16279.127578735352
26 17545.904678344727
27 17681.908630371094
28 17966.967163085938
29 18367.57339477539
30 18949.206939697266
31 20058.462310791016
32 20515.96841430664
33 20611.398696899414
34 21263.752700805664
35 22583.10792541504
36 22899.3370513916
37 23757.247817993164
38 24102.850296020508
39 24306.790145874023
40 24668.67726135254
41 25048.823318481445
42 25966.328018188477
43 26238.588272094727
44 26710.203323364258
45 26901.019760131836
46 27399.823318481445
47 

In [14]:
print(mae/len(img_paths)) # obtain MAE

524.0554501879346


### Visualize the prediction output